## Day and Woodward

Avec ce problème nous étudions les solveurs `hlld` et `fivewaves` de `fv2d`, et les comparons au solveur `hlld` de Athena++, combiné à un intégrateur Van Leer de second ordre. Mis à part les solveurs, nous utiliserons une grille de 512 points et une CFL de 0,4. La reconstruction spatiale utilisée par défaut sera une _PLM_, combinée à un schéma temporel _Runge-Kutta_ d'ordre 2. On note que la divergence de $\vec{B}$ ne pose pas de problème en 1D et ne sera pas étudiée ici.
```
    const real_t B0 = 1.0 / std::sqrt(4 * M_PI);

    if (x < midbox) {
      Q(j, i, IR) = 1.08;
      Q(j, i, IU) = 1.2;
      Q(j, i, IV) = 0.01;
      Q(j, i, IW) = 0.5;
      Q(j, i, IP) = 0.95;
      Q(j, i, IBX) = B0 * 4.0;
      Q(j, i, IBY) = B0 * 3.6;
      Q(j, i, IBZ) = B0 * 2.0;
    }
    else {
      Q(j, i, IR) = 1.0;
      Q(j, i, IU) = 0.0;
      Q(j, i, IV) = 0.0;
      Q(j, i, IW) = 0.0;
      Q(j, i, IP) = 1.0;
      Q(j, i, IBX) = B0 * 4.0;
      Q(j, i, IBY) = B0 * 4.0;
      Q(j, i, IBZ) = B0 * 2.0;
```

In [ ]:
print(b0*4)
print(b0*3.6)
print(b0*2)

### Problem Set Up

In [ ]:
from datahandler import Fv2dCode, AthenaCode, IniFile
from pathlib import Path
home = Path().home() / "Documents"

In [ ]:
problem_name = "dai_woodward"

In [ ]:
ini_file = IniFile(home/f"fv2d/settings/{problem_name}.ini")
ini_file.set_param("mesh", "Nx", 512)
ini_file.set_param("mesh", "Ny", 10)
ini_file.set_param("solvers", "CFL", 0.4)
ini_file.set_param("physics", "gamma0", 5/3)

# HLLD solver + RK2
ini_file.set_param("solvers", "riemann_solver", "hlld")
ini_file.write(f"{problem_name}_dc.ini")

# HLLD solver + RK2 without div cleaning
ini_file.set_param("solvers", "riemann_solver", "hlld")
ini_file.set_param("solvers", "div_cleaning", "none")
ini_file.write(f"{problem_name}_nodc.ini")

# Five waves solver + RK2 and div cleaning
ini_file.set_param("solvers", "riemann_solver", "fivewaves")
ini_file.write(f"{problem_name}_5w.ini")

# Five waves solver + Euler time stepping and div cleaning
ini_file.set_param("solvers", "riemann_solver", "fivewaves")
ini_file.set_param("solvers", "time_stepping", "euler")
ini_file.write(f"{problem_name}_5w_euler.ini")

# HLLD solver + Euler time stepping
ini_file.set_param("solvers", "riemann_solver", "hlld")
ini_file.set_param("solvers", "time_stepping", "euler")
ini_file.set_param("solvers", "div_cleaning", "none")
ini_file.write(f"{problem_name}_euler.ini")

In [ ]:
# Compile and run the code
fv2d = Fv2dCode(base_path=home/"fv2d")
#fv2d.compile()
fv2d.run(inifile=f"{problem_name}_dc.ini")
fv2d.run(inifile=f"{problem_name}_nodc.ini")
fv2d.run(inifile=f"{problem_name}_5w.ini")
fv2d.run(inifile=f"{problem_name}_5w_euler.ini")
fv2d.run(inifile=f"{problem_name}_euler.ini")

In [ ]:
ini_file = IniFile(home/"athena/inputs/mhd/athinput.rj2a", athena_fmt=True)
ini_file.set_param("mesh", "nx1", 512)
ini_file.set_param("time", "cfl_number", 0.4)
ini_file.set_param("time", "tlim", 0.2)
ini_file.set_param("mesh", "x1max", 1.1)
ini_file.set_param("mesh", "x1min", 0.0)
ini_file.set_param("problem", "xshock", 0.55) # position de l'interface initiale
b0 = 1/np.sqrt(4*np.pi)
ini_file.set_param("problem", "bxl", f"{b0*4:.16f}") # athena prend b0*2 pour Bx
ini_file.set_param("problem", "bxr", f"{b0*4:.16f}")
ini_file.set_param("job", "problem_id", "DaiWoodward")
ini_file.write("athinput.rj2a")

In [ ]:
athena = AthenaCode(home/"athena")
athena.compile(ini_file)
athena.run(inifile="athinput.rj2a")

### Data Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datahandler import AthenaTabular, Fv2DH5
from pathlib import Path

In [ ]:
athena_data_path = Path("athena_output")
athena_data_file = list(sorted(athena_data_path.glob("*.tab")))[-1]
athena_data = AthenaTabular(athena_data_file)

In [ ]:
# hlld + RK2 Data
fv2d_data_path = Path("fv2d_output")
fv2d_data = Fv2DH5(fv2d_data_path/f"run_{problem_name}_dc.h5", reshape=True)
fv2d_data_5w = Fv2DH5(fv2d_data_path/f"run_{problem_name}_5w.h5", reshape=True) # fv2d outputs are 2D; we select only a slice along x
fv2d_data_5we = Fv2DH5(fv2d_data_path/f"run_{problem_name}_5w_euler.h5", reshape=True) # fv2d outputs are 2D; we select only a slice along x
fv2d_data_e = Fv2DH5(fv2d_data_path/f"run_{problem_name}_euler.h5", reshape=True) # fv2d outputs are 2D; we select only a slice along x
fv2d_data_nodc = Fv2DH5(fv2d_data_path/f"run_{problem_name}_nodc.h5", reshape=True) # fv2d outputs are 2D; we select only a slice along x

In [ ]:
def plot_attrs(attr, *, longname=None, title="Dai-Woodward Test", savetowiki=False) -> None:
    """ Plot the given attribute for all the data sets.
    Parameters
    ----------
    attr : str
        The attribute to plot.
    longname : str
        The long name of the attribute to plot, used for the $y$-axis.
        If None, the attribute name will be used.
    """
    plt.figure(figsize=(10, 10))
    plt.plot(athena_data.x, getattr(athena_data, attr), '--k', label="Athena-hlld-vl2")
    plt.plot(fv2d_data.x, getattr(fv2d_data, attr), '--', label="FV2D-hlld-RK2")
    plt.plot(fv2d_data_5w.x, getattr(fv2d_data_5w, attr), '--', label="FV2D-5waves-RK2")
    plt.plot(fv2d_data_5we.x, getattr(fv2d_data_5we, attr), '--', label="FV2D-5waves-Euler")
    plt.plot(fv2d_data_e.x, getattr(fv2d_data_e, attr), '--', label="FV2D-hlld-Euler")
    plt.plot(fv2d_data_nodc.x, getattr(fv2d_data_nodc, attr), '--', label="FV2D-hlld-noDC")
    plt.title(title)
    if longname is not None:
        plt.ylabel(longname)
    else:
        plt.ylabel(attr)
    plt.xlabel("x")
    plt.legend()
    plt.grid()
    if savetowiki:
        plt.savefig(home/ f"fvNd-unit-test.wiki/imgs/{problem_name}_{attr}.png")
    plt.show()

In [ ]:
save = True
plot_attrs("rho", longname=r"$\rho$", savetowiki=save)
plot_attrs("u", longname=r"$v_x$", savetowiki=save)
plot_attrs("v", longname=r"$v_y$", savetowiki=save)
plot_attrs("w", longname=r"$v_z$", savetowiki=save)
plot_attrs("p", longname=r"$P$", savetowiki=save)
#plot_attrs("Bx", longname=r"$B_x$", savetowiki=save)
plot_attrs("By", longname=r"$B_y$", savetowiki=save)
plot_attrs("Bz", longname=r"$B_z$", savetowiki=save)

### Error Analysis

In [ ]:
from typing import Callable


def absolute_error(y: np.ndarray, y_ref: np.ndarray, epsilon=1e-3) -> np.ndarray:
    """ Compute the absolute error between two arrays.
    Parameters
    ----------
    y : np.ndarray
        The data to compare.
    y_ref : np.ndarray
        The reference data.
    Returns
    -------
    np.ndarray
        The absolute error between the two arrays.
    """
    return np.abs(y - y_ref)


def relative_error(y: np.ndarray, y_ref: np.ndarray, *, mixed_err=True, epsilon=1) -> np.ndarray:
    """ Compute the relative error between two arrays.
    Parameters
    ----------
    y : np.ndarray
        The data to compare.
    y_ref : np.ndarray
        The reference data.
    mixed_err : bool
        Allow usage of a absolute error on zones where values are less than 1.
    Returns
    -------
    np.ndarray
        The relative error between the two arrays.
    """
    abs_err = absolute_error(y, y_ref)
    if mixed_err:
        not_small_values = y_ref >= epsilon #and y > epsilon
        abs_err[not_small_values] /= np.abs(y_ref[not_small_values])
        return abs_err
        #return np.where(np.abs(y_ref) < epsilon, absolute_error(y, y_ref), np.abs(y - y_ref) / np.abs(y_ref))
    return abs_err / np.abs(y_ref)

def plot_error(attr: str, *, longname: str=None, error_type: Callable = relative_error, norm_ord=2, epsilon=1, scale='log', savetowiki=False) -> None:
    """ Plot the error between the data sets.
    Parameters
    ----------
    attr : str
        The attribute to plot.
    longname : str
        The long name of the attribute to plot, used for the $y$-axis.
        If None, the attribute name will be used.
    error_type : Callable
        The error function to use. Can be either absolute_error or relative_error.
    norm_ord : int
        The order of the norm to use. Default is 2.
    """

    plt.figure(figsize=(10, 10))
    e1 = error_type(getattr(fv2d_data, attr), getattr(athena_data, attr), epsilon=epsilon)
    e2 = error_type(getattr(fv2d_data_5w, attr), getattr(athena_data, attr), epsilon=epsilon)
    e3 = error_type(getattr(fv2d_data_5we, attr), getattr(athena_data, attr), epsilon=epsilon)
    e4 = error_type(getattr(fv2d_data_e, attr), getattr(athena_data, attr), epsilon=epsilon)
    e5 = error_type(getattr(fv2d_data_nodc, attr), getattr(athena_data, attr), epsilon=epsilon)

    plt.plot(fv2d_data.x, e1, '--', label=f"FV2D-hlld-RK2, L{norm_ord}-norm= {np.linalg.norm(e1, ord=norm_ord)/len(e1):.2e}")
    plt.plot(fv2d_data.x, e2, '--', label=f"FV2D-5waves-RK2, L{norm_ord}-norm= {np.linalg.norm(e2, ord=norm_ord)/len(e2):.2e}")
    plt.plot(fv2d_data.x, e3, '--', label=f"FV2D-5waves-Euler, L{norm_ord}-norm= {np.linalg.norm(e3, ord=norm_ord)/len(e3):.2e}")
    plt.plot(fv2d_data.x, e4, '--', label=f"FV2D-hlld-Euler, L{norm_ord}-norm= {np.linalg.norm(e4, ord=norm_ord)/len(e4):.2e}")
    plt.plot(fv2d_data.x, e5, '--', label=f"FV2D-hlld-noDC, L{norm_ord}-norm= {np.linalg.norm(e5, ord=norm_ord)/len(e5):.2e}")
    plt.title(f"Dai-Woodward - {error_type.__name__.replace('_', ' ').title()}")
    ymax = np.max(np.array([e1.max(), e2.max(), e3.max(), e4.max(), e5.max()]))
    plt.ylim(1e-7, ymax)
    if longname is not None:
        plt.ylabel(longname)
    else:
        plt.ylabel(attr)
    plt.xlabel("x")
    plt.yscale(scale)
    plt.legend()
    plt.grid()
    if savetowiki:
        plt.savefig(home/ f"fvNd-unit-test.wiki/imgs/{problem_name}_err_{attr}.png")
    plt.show()

In [ ]:
save = True
plot_error("rho", longname=r"$\rho$", error_type=absolute_error, savetowiki=save)
plot_error("u", longname=r"$v_x$", error_type=absolute_error, savetowiki=save)
plot_error("v", longname=r"$v_y$", error_type=absolute_error, savetowiki=save)
plot_error("w", longname=r"$v_z$", error_type=absolute_error, savetowiki=save)
plot_error("p", longname=r"$P$", error_type=absolute_error, savetowiki=save)
plot_error("By", longname=r"$B_y$", error_type=absolute_error, savetowiki=save)
plot_error("By", longname=r"$B_y$", error_type=absolute_error, savetowiki=save)

### Convergence Analysis

Le but est ici de tester la convergence de nos différents solveurs selon la taille du maillage, i.e. le nombre de cellules. Nous choissisons de tester chaque solveurs pour les valeurs suivantes $Nx=32, 64, 128 , 256, 512$

In [ ]:
from datahandler import Fv2dCode, IniFile, AthenaCode
from pathlib import Path
home = Path().home() / "Documents"

In [ ]:
# generate all the ini_files for the different tests
original_ini = home/f"fv2d/settings/{problem_name}.ini"
athena_ini = home/"athena/inputs/mhd/athinput.rj2a"
for i in range(5, 10):
    at_ini = IniFile(athena_ini, athena_fmt=True)
    at_ini.set_param("mesh", "nx1", 2**i)
    at_ini.set_param("time", "cfl_number", 0.4)
    at_ini.set_param("time", "tlim", 0.2)
    at_ini.set_param("mesh", "x1max", 1.1)
    at_ini.set_param("mesh", "x1min", 0.0)
    at_ini.set_param("problem", "xshock", 0.55) # position de l'interface initiale
    b0 = 1/np.sqrt(4*np.pi)
    at_ini.set_param("problem", "bxl", f"{b0*4:.16f}") # athena prend b0*2 pour Bx
    at_ini.set_param("problem", "bxr", f"{b0*4:.16f}")
    at_ini.set_param("job", "problem_id", "DaiWoodward")
    at_ini.write(f"athinput_{2**i:04d}.rj2a")
    for solver in ["hlld", "fivewaves"]:
        for time_stepping in ["RK2", "euler"]:
            new_ini = f"{problem_name}_{2**i:04d}_{solver}_{time_stepping}.ini"
            ini = IniFile(original_ini)
            ini.set_param("mesh", "Nx", 2**i)
            ini.set_param("mesh", "Ny", 1)
            ini.set_param("solvers", "riemann_solver", solver)
            ini.set_param("solvers", "time_stepping", time_stepping)
            ini.set_param("solvers", "div_cleaning", "none")
            ini.set_param("solvers", "CFL", 0.4)
            ini.set_param("physics", "gamma0", 5/3)
            ini.write(new_ini)

In [ ]:
# Here we assume (we know) that the fv2d code is already compiled
# and that the athena code is already compiled
ath_output = Path('athena_output')
for item in Path().glob("athinput_*.rj2a"):
    athena = AthenaCode(home/"athena")
    this_output = ath_output / item.stem.replace("athinput_", "")
    this_output.mkdir(exist_ok=True, parents=True)
    athena.run(inifile=item.name, destination=this_output)

In [ ]:
fv2d = Fv2dCode(base_path=home/"fv2d")
# fv2d.compile()
fv2d_output = Path('fv2d_output')
files = list(sorted(Path().glob(f"{problem_name}_*.ini")))
for i, item in enumerate(files):
    nx = item.stem.split("_")[2]
    this_output = fv2d_output / nx
    this_output.mkdir(exist_ok=True, parents=True)
    fv2d.run(inifile=item.name, destination=this_output)

In [ ]:
from datahandler import Fv2DH5, AthenaTabular
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
athena_data = {}
fv2d_data = {}
errors = {"-".join(algo.split('_')[:-1]): [] for algo in fv2d_data}
for i in range(5, 10):
    dirname = Path("athena_output") / f"{2**i:04d}"
    item = list(sorted(dirname.glob("*.tab")))[-1]
    athena_data[f"{2**i:04d}"] = AthenaTabular(item)
    athena_data[f"{2**i:04d}"].rescale() # by default on AThena is [-0.5, 0.5] and on fv2d is [0, 1]
    item = list(sorted(Path(f"fv2d_output/{2**i:04d}").glob("*.h5")))
    fv2d_data[f"{2**i:04d}"] = [(str(it.stem), Fv2DH5(it, reshape=True)) for it in item]

In [ ]:
errors = {"-".join(algo[0].split('_')[-2:]):[] for algo in list(fv2d_data.values())[0]}
for i in range(5, 10):
    nx = f"{2**i:04d}"
    for algo, val in fv2d_data[nx]:
        algo = "-".join(algo.split('_')[-2:]) # on veut rassembler les algo
        err = np.linalg.norm(val.rho - athena_data[nx].rho, ord=2)/int(nx)
        print(f"error {algo} = {err:.2e}")
        errors[algo].append(err)

In [ ]:
plt.figure(figsize=(10, 10))
nx_values = np.array([2**i for i in range(5, 10)])
for algo, err in errors.items():
    m, b = np.polyfit(np.log(nx_values), np.log(err), deg=1)
    plt.plot(nx_values, err, '--o', label=fr"{algo}: $\log(err)={m:.2f}N_x{b:.2f}$")
plt.title("Dai and Woodward: Error Convergence")

O_x = errors["fivewaves-RK2"][0] * (nx_values[0] / nx_values)
O_x2 = errors["fivewaves-RK2"][0] * (nx_values[0] / nx_values)**2  
plt.plot(nx_values, O_x, ':', label=r"$\mathcal{O}(\Delta x)$")
plt.plot(nx_values, O_x2, ':', label=r"$\mathcal{O}(\Delta x^2)$")

plt.text(nx_values[-1], O_x2[-1], r"$\Delta x^2$", fontsize=12)
plt.text(nx_values[-1], O_x[-1], r"$\Delta x$", fontsize=12)
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r"$N_x$")
plt.ylabel(r"$L^2$ error")
plt.legend()
plt.grid()
plt.show()